# Script for creating animation keyframes of warehouse growth in San Bernardino and Riverside Counties, California
#### Created for the Robert Redford Conservancy for Southern California Sustainability in 2022
Authored by Graham E. Brady, gbrady@pitzer.edu

#### We began by importing the required libraries, setting the global parameters, and writing some helper functions.

In [1]:
import arcpy
import os
import numpy as np
import pandas as pd
import zipfile
import requests
ListOfYears = ['1975','1980','1985','1990','1995','2000','2005','2010','2015','2020']

data_path = r'D:\01_OntarioWarehouses\InlandEmpireWarehouses\data'
dst_path = r'D:\01_OntarioWarehouses\InlandEmpireWarehouses\maps'

def unique_values(table, field):  ##uses list comprehension
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted(filter(None, {row[0] for row in cursor}))

def addCommas(number):
    return ("{:,}".format(number))

def updateNorthArrow(logo,northArrow):
    if logo.visible:
        northArrow.elementPositionX = 3.27
        northArrow.elementPositionY = 7.83
    else:
        northArrow.elementPositionX = 0.67
        northArrow.elementPositionY = 7.83

#### Then, parcel data from San Bernardino and Riverside Counties were downloaded from their respective online repositories.

In [3]:
#San Bernardino County
from ftplib import FTP

os.chdir(data_path + r'\raw')

#San Bernardino County
FTP_host = 'gis1.sbcounty.gov'
FTP = FTP()
FTP.connect(FTP_host)
FTP.login(user='gisftp',passwd='1sbcftp1')
#FTP.dir()
path = 'gims/parcel_basemap_data/'
FTP.cwd(path)
filename = 'countywide_parcels_04_18_2022.zip'
#/countywide_parcels_02_28_2022.zip' and other similar filenames
FTP.retrbinary("RETR " + filename, open('SB_' + filename, 'wb').write)
filename = 'Assessor%20Use%20Codes%2005-21-2012.xls'
FTP.retrbinary("RETR " + filename, open('SB_' + filename, 'wb').write)
FTP.quit()
print('SB County parcel data were downloaded.')

#Riverside County
url = 'https://gis2.rivco.org/Portals/0/Documents/downloads/ParcelAttributed0422.zip'
r = requests.get(url, allow_redirects=True)
open('riverside_parcel_raw_data.zip', 'wb').write(r.content)
print('Riverside County parcel data were downloaded.')
#
#Imperial County
#ic = 'https://services1.arcgis.com/fwUrSNrE506Uxp7v/arcgis/rest/services/Parcels_APN_Only_new/FeatureServer/0'
#arcpy.conversion.WFSToFeatureClass(input_WFS_server, WFS_feature_type, out_path, out_name, {is_complex}, {max_features}, {expose_metadata}, {swap_xy})
#
#Los Angeles County
#

11-13-14  03:34PM       <DIR>          gims


182174101

#### Some additional files are required to attribute parcel data from Riverside County's assessor office. All of the files need to be unzipped.

In [57]:
url = 'https://gis2.rivco.org/Portals/0/Documents/downloads/Assessor_Tables0422.zip'
r = requests.get(url, allow_redirects=True)
open('riverside_assessor_tables.zip', 'wb').write(r.content)
print('Riverside County assessor data were downloaded.')
with zipfile.ZipFile("riverside_assessor_tables.zip","r") as zip_ref:
    zip_ref.extractall("Riverside")
    print('Extracted Riverside parcel data.')

Riverside County assessor data were downloaded.
Extracted Riverside parcel data.


In [9]:
#Unzipping raw data
with zipfile.ZipFile("SB_countywide_parcels_04_18_2022.zip","r") as zip_ref:
    zip_ref.extractall("SanBernardino")
    print('Extracted San Bernardino parcel data.')
    
with zipfile.ZipFile("riverside_parcel_raw_data.zip","r") as zip_ref:
    zip_ref.extractall("Riverside")
    print('Extracted Riverside parcel data.')


### Starting with San Bernardino County, we set the relevant layout items in the ArcGIS environment, subselect the warehouses within the parcel shapefile, calculate the growth of warehouses by year, and write map files for each year in the analysis as well as the present. We include parcels whose years cannot be attributed in the final static map of the current distribution.

#### In the first published iteration of this map and workflow, the keyword "warehouse" was used to classify parcels for area and count calculations and map creation.

In [12]:
######Starting with SB county#######
arcpy.env.workspace =r'D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\temp.gdb'
arcpy.env.overwriteOutput = True
os.chdir(data_path + r'\raw')

aprx = arcpy.mp.ArcGISProject("Current")
lyt = aprx.listLayouts("LayoutSanBernardino")[0]
m = aprx.listMaps("WarehousesSanBernardino")[0]
m.addDataFromPath(r'D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\raw\SanBernardino\pbmpoly.shp')

ListOfYears = ['1975','1980','1985','1990','1995','2000','2005','2010','2015','2020']
lytText = lyt.listElements("TEXT_ELEMENT", "YearsDisplayedText")[0]

##Layout Objects###
lytText = lyt.listElements("TEXT_ELEMENT", "YearsDisplayedText")[0]
lytTextTotalArea = lyt.listElements("TEXT_ELEMENT", "TotalArea")[0]
lytTextDisclaimer = lyt.listElements("TEXT_ELEMENT", "disclaimer")[0]
lytTextWarehouseCount = lyt.listElements("TEXT_ELEMENT", "WarehouseCount")[0]

In [13]:
####San Bernardino Warehouse Subselection####
codes = pd.read_excel('Assessor Use Codes 05-21-2012.xls')
warehouse_codes = codes[codes['Description'].str.lower().str.contains('warehouse',na=False)]
parcel_types = warehouse_codes['Use Code'].tolist()

In [14]:
parcels = m.listLayers('pbmpoly')[0]
for parcel_type in parcel_types:
    query = "TYPEUSE = " + str(parcel_type)
    arcpy.management.SelectLayerByAttribute(parcels,"ADD_TO_SELECTION",query)
arcpy.management.CopyFeatures(parcels, r'\derived\Warehouses_SanBernardino.shp')
arcpy.SelectLayerByAttribute_management(parcels, "CLEAR_SELECTION")

lyrList = m.listLayers('Warehouses_SanBernardino')
warehouseLyr = lyrList[0]
warehouseLyr.name = 'Warehouses'
sym = warehouseLyr.symbology
sym.renderer.symbol.color = {'CMYK' : [0, 50, 100, 0, 100]}
sym.renderer.symbol.outlineColor = {'CMYK' : [0, 50, 100, 0, 0]}
warehouseLyr.symbology = sym

m.removeLayer(m.listLayers('pbmpoly')[0])

In [15]:
###Calculate total area and count of warehouses for the county###
warehouseLyr.definitionQuery = None
areaSB = sum([row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "Shape_Area")])
areaSB = round(areaSB)
countSB = arcpy.management.GetCount(warehouseLyr)
print('{} has {} records covering ~{} square feet of land in SB County.'.format(warehouseLyr, countSB[0], addCommas(areaSB)))

###Calculate SB County Areas by years###
OutList = []
totalCount = 0
sumAreas = 0

for year in ListOfYears:
    arcpy.management.SelectLayerByAttribute(warehouseLyr, 'NEW_SELECTION', "BASE_YEAR <= " + year + " And BASE_YEAR <> 0")
    #warehouseLyr.definitionQuery = "BASE_YEAR <= " + year + " And BASE_YEAR <> 0"
    areas = [row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "Shape_Area")]
    sumAreas = sum(areas)
    totalCount = len(areas)
    print('calculating area and count by year for SB county in year ', year)
    OutList.append([year, sumAreas, totalCount])
    #Calculate square footage and total number of warehouses in 
    #each city of San Bernardino County for all years of study
    #Do not display on map
    #percentage of total warehouses
    
arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
    
AreaByYear = np.array(OutList)
struct_array = numpy.core.records.fromarrays(
    AreaByYear.transpose(), numpy.dtype([('Year', '<i8'), ('AreaTotal', '<f8'), ('WarehousesCount', '<i8')]))
arcpy.da.NumPyArrayToTable(struct_array,r"D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\temp.gdb\AreaCountByYearSanBernardino")
m.addDataFromPath(data_path + r'\temp.gdb\AreaCountByYearSanBernardino')
AreaTable = m.listTables("AreaCountByYearSanBernardino*")[0]
delimfield = arcpy.AddFieldDelimiters(AreaTable, 'Year')

Warehouses has 3351 records covering ~765,763,495 square feet of land in SB County.
calculating area and count by year for SB county in year  1975
calculating area and count by year for SB county in year  1980
calculating area and count by year for SB county in year  1985
calculating area and count by year for SB county in year  1990
calculating area and count by year for SB county in year  1995
calculating area and count by year for SB county in year  2000
calculating area and count by year for SB county in year  2005
calculating area and count by year for SB county in year  2010
calculating area and count by year for SB county in year  2015
calculating area and count by year for SB county in year  2020


In [16]:
for year in ListOfYears:
    warehouseLyr.definitionQuery = "BASE_YEAR <= " + year + " And BASE_YEAR <> 0"
    with arcpy.da.SearchCursor(AreaTable, 'AreaTotal', where_clause=delimfield +' = ' + str(year)) as cursor:
        for row in cursor:
            totalArea = row[0]
            totalArea = round(totalArea)
    with arcpy.da.SearchCursor(AreaTable, 'WarehousesCount', where_clause=delimfield +' = ' + str(year)) as cursor:
        for row in cursor:
            warehousesCount = row[0]
    lytText.text = 'Distribution in ' + str(year)
    lytTextTotalArea.text = 'Area Occupied by Warehouses: '+ str(addCommas(totalArea)) + ' ft<sup>2</sup>'
    lytTextWarehouseCount.text = 'Total Number of Warehouses: ' + str(addCommas(warehousesCount))
    print('assessing year: ', year)
    arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
    lyt.exportToPDF(dst_path + r'\SBwarehouses' + str(year)) #PNG or SVG
    lyt.exportToSVG(dst_path + r'\SBwarehouses' + str(year)) #PNG or SVG
    lyt.exportToPNG(dst_path + r'\SBwarehouses' + str(year)) #PNG or SVG
arcpy.Delete_management(data_path + r'\temp.gdb\AreaCountByYearSanBernardino')
warehouseLyr.definitionQuery = None
lytText.text = 'Distribution in 2021'
lytTextTotalArea.text = 'Area Occupied by Warehouses: '+ str(addCommas(round(float(areaSB)))) + ' ft<sup>2</sup>'
lytTextWarehouseCount.text = 'Total Number of Warehouses: ' + str(addCommas(int(countSB[0])))
print('assessing SB County all entries.')
lyt.exportToPDF(dst_path + r'\SBwarehouses2021') #PNG or SVG
lyt.exportToSVG(dst_path + r'\SBwarehouses2021') #PNG or SVG
lyt.exportToPNG(dst_path + r'\SBwarehouses2021') #PNG or SVG


assessing year:  1975
assessing year:  1980
assessing year:  1985
assessing year:  1990
assessing year:  1995
assessing year:  2000
assessing year:  2005
assessing year:  2010
assessing year:  2015
assessing year:  2020
assessing SB County all entries.


'D:\\01_OntarioWarehouses\\InlandEmpireWarehouses\\maps\\SBwarehouses2021.png'

![San Bernardino County Warehouses in 2021](SBwarehouses2021.svg)

### For Riverside County, we set the relevant layout items in the ArcGIS environment, subselect the warehouses within the parcel shapefile, and merge the parcel shapefile to the related assessor table of parcel characteristics. Using the shared APNs (assessor parcel number), we add the base year of each parcel in the assessor records to our database.

#### In the first published iteration of this map and workflow, the keyword "warehouse" was used to classify parcels for area and count calculations and map creation.

In [5]:
######Riverside County preliminary connections#####
#arcpy.env.workspace = r'D:\01_OntarioWarehouses\WarehouseStudy\Riverside GIS Data\RiversideWarehouses\RiversideAnimation.gdb'

aprx = arcpy.mp.ArcGISProject("Current")
lyt = aprx.listLayouts("LayoutRiverside")[0]
m = aprx.listMaps("WarehousesRiverside")[0]
lytText = lyt.listElements("TEXT_ELEMENT", "YearsDisplayedText")[0]
lytTextTotalArea = lyt.listElements("TEXT_ELEMENT", "TotalArea")[0]
lytTextDisclaimer = lyt.listElements("TEXT_ELEMENT", "disclaimer")[0]
lytTextWarehouseCount = lyt.listElements("TEXT_ELEMENT", "WarehouseCount")[0]

In [7]:
######Riverside Parcel Subselection Process#####
m.addDataFromPath(data_path + r'\raw\Riverside\ParcelAttributed.gdb\PARCELS_CREST')
parcels = m.listLayers('PARCELS_CREST')[0]
parcel_types = pd.DataFrame(unique_values(parcels, 'CLASS_CODE'))
warehouse_codes = parcel_types[parcel_types[0].str.lower().str.contains('warehouse')]
for parcel_type in warehouse_codes[0].tolist():
    query = "CLASS_CODE = " + "'" + str(parcel_type) + "'"
    arcpy.management.SelectLayerByAttribute(parcels,"ADD_TO_SELECTION",query)
arcpy.management.CopyFeatures(parcels, data_path + r'\derived\Warehouses_Riverside_prelim.shp')
arcpy.SelectLayerByAttribute_management(parcels, "CLEAR_SELECTION")
m.removeLayer(m.listLayers('PARCELS_CREST')[0])
warehouse_codes

,0
97,CT-Warehouse/Cold Storage
98,CT-Warehouse/Dry Storage
255,Warehouse/Dry Storage
256,Warehouse/Mega


In [8]:
####Joining the characteristics table to the parcel shapefile to include yearly growth in the analysis###
warehouseLyr = m.listLayers('Warehouses*')[0]

m.addDataFromPath(data_path + r'\raw\Riverside\CREST_Tables.gdb\CREST_PROPERTY_CHAR')
arcpy.management.AddJoin(warehouseLyr, "APN", 
                         "CREST_PROPERTY_CHAR", "PIN", 
                         "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
arcpy.conversion.FeatureClassToFeatureClass("Warehouses_Riverside_prelim", r"D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\derived", "Warehouses_Riverside.shp", '', 'APN "APN" true true false 32 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.APN,0,32;FLAG "FLAG" true true false 2 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.FLAG,0,2;MAIL_STREE "MAIL_STREE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.MAIL_STREE,0,64;MAIL_CITY "MAIL_CITY" true true false 48 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.MAIL_CITY,0,48;SITUS_STRE "SITUS_STRE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.SITUS_STRE,0,64;SITUS_CITY "SITUS_CITY" true true false 48 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.SITUS_CITY,0,48;STREET_NUM "STREET_NUM" true true false 10 Long 0 10,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.STREET_NUM,-1,-1;STREET_PRE "STREET_PRE" true true false 2 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.STREET_PRE,0,2;STREET_NAM "STREET_NAM" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.STREET_NAM,0,64;STREET_TYP "STREET_TYP" true true false 8 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.STREET_TYP,0,8;STREET_SUF "STREET_SUF" true true false 16 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.STREET_SUF,0,16;UNIT_NUMBE "UNIT_NUMBE" true true false 8 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.UNIT_NUMBE,0,8;CITY "CITY" true true false 32 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.CITY,0,32;ZIP_CODE "ZIP_CODE" true true false 16 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.ZIP_CODE,0,16;CLASS_CODE "CLASS_CODE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.CLASS_CODE,0,64;MULTIPLE "MULTIPLE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.MULTIPLE,0,64;SUBDIVISIO "SUBDIVISIO" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.SUBDIVISIO,0,64;ACREAGE "ACREAGE" true true false 19 Double 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.ACREAGE,-1,-1;RECORDER_M "RECORDER_M" true true false 16 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.RECORDER_M,0,16;BOOK "BOOK" true true false 4 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.BOOK,0,4;PAGE "PAGE" true true false 4 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.PAGE,0,4;MAP_BOOK_P "MAP_BOOK_P" true true false 16 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.MAP_BOOK_P,0,16;COUNTY_COD "COUNTY_COD" true true false 2 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.COUNTY_COD,0,2;LOT_TYPE "LOT_TYPE" true true false 16 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.LOT_TYPE,0,16;LOT "LOT" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.LOT,0,64;BLOCK "BLOCK" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.BLOCK,0,64;CAME_FROM "CAME_FROM" true true false 254 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.CAME_FROM,0,254;TAX_RATE_A "TAX_RATE_A" true true false 6 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.TAX_RATE_A,0,6;LAND "LAND" true true false 19 Double 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.LAND,-1,-1;STRUCTURES "STRUCTURES" true true false 19 Double 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.STRUCTURES,-1,-1;PRIMARY_OW "PRIMARY_OW" true true false 128 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.PRIMARY_OW,0,128;ALL_OWNER_ "ALL_OWNER_" true true false 254 Text 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.ALL_OWNER_,0,254;SHAPE_Leng "SHAPE_Leng" true true false 19 Double 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.SHAPE_Leng,-1,-1;SHAPE_Area "SHAPE_Area" true true false 19 Double 0 0,First,#,Warehouses_Riverside_prelim,Warehouses_Riverside_prelim.SHAPE_Area,-1,-1;OBJECTID "OBJECTID" false true false 4 Long 0 9,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.OBJECTID,-1,-1;PIN "PIN" true true false 32 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.PIN,0,32;BUILDING_ID "BUILDING_ID" true true false 4 Long 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.BUILDING_ID,-1,-1;STREET_SURFACED "STREET_SURFACED" true true false 32 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.STREET_SURFACED,0,32;YEAR_BUILT "YEAR_BUILT" true true false 4 Long 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.YEAR_BUILT,-1,-1;NUMBER_OF_STORIES "NUMBER_OF_STORIES" true true false 4 Long 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.NUMBER_OF_STORIES,-1,-1;ROOF_TYPE "ROOF_TYPE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.ROOF_TYPE,0,64;GARAGE_TYPE "GARAGE_TYPE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.GARAGE_TYPE,0,64;GARAGE_SIZE "GARAGE_SIZE" true true false 8 Double 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.GARAGE_SIZE,-1,-1;CARPORT_SIZE "CARPORT_SIZE" true true false 8 Double 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.CARPORT_SIZE,-1,-1;HAS_FIREPLACE "HAS_FIREPLACE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.HAS_FIREPLACE,0,64;BATH_COUNT "BATH_COUNT" true true false 4 Long 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.BATH_COUNT,-1,-1;CENTRAL_HEATING "CENTRAL_HEATING" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.CENTRAL_HEATING,0,64;CENTRAL_COOLING "CENTRAL_COOLING" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.CENTRAL_COOLING,0,64;DESIGN_TYPE "DESIGN_TYPE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.DESIGN_TYPE,0,64;CONSTRUCTION_TYPE "CONSTRUCTION_TYPE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.CONSTRUCTION_TYPE,0,64;QUALITY_CODE "QUALITY_CODE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.QUALITY_CODE,0,64;SHAPE_CODE "SHAPE_CODE" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.SHAPE_CODE,0,64;LIVING_AREA "LIVING_AREA" true true false 8 Double 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.LIVING_AREA,-1,-1;ACTUAL_AREA "ACTUAL_AREA" true true false 8 Double 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.ACTUAL_AREA,-1,-1;HAS_POOL "HAS_POOL" true true false 8 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.HAS_POOL,0,8;BEDROOM_COUNT "BEDROOM_COUNT" true true false 4 Long 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.BEDROOM_COUNT,-1,-1;FAIRWAY "FAIRWAY" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.FAIRWAY,0,64;WATERFRONT "WATERFRONT" true true false 64 Text 0 0,First,#,Warehouses_Riverside_prelim,CREST_PROPERTY_CHAR.WATERFRONT,0,64', '')
m.removeLayer(m.listLayers('Warehouses_Riverside_prelim*')[0])
warehouseLyr = m.listLayers('Warehouses*')[0]




In [9]:
#Updating symbology of the warehouse layer
warehouseLyr.name = 'Warehouses'
sym = warehouseLyr.symbology
sym.renderer.symbol.color = {'CMYK' : [0, 50, 100, 0, 100]}
sym.renderer.symbol.outlineColor = {'CMYK' : [0, 50, 100, 0, 0]}
warehouseLyr.symbology = sym

In [10]:
###Calculating total area and count of warehouses for the county###
arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
warehouseLyr.definitionQuery = None

areaRiv = sum([row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "SHAPE_Area")])
areaRiv = round(areaRiv)
countRiv = arcpy.management.GetCount(warehouseLyr)
print('{} has {} records covering ~{} square feet of land in Riverside County.'.format(warehouseLyr, countRiv[0], areaRiv))

warehouseLyr.definitionQuery = "YEAR_BUILT = 0"
areaRivZero = sum([row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "SHAPE_Area")])
areaRivZero = round(areaRivZero)
countRivZero = arcpy.management.GetCount(warehouseLyr)
warehouseLyr.definitionQuery = None

OutList = []
totalCount = 0
sumAreas = 0

for year in ListOfYears:
    arcpy.management.SelectLayerByAttribute(warehouseLyr, 'NEW_SELECTION', "YEAR_BUILT <= " + year + " And YEAR_BUILT <> 0 And YEAR_BUILT <> 1776")
    areas = [row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "SHAPE_Area")]
    sumAreas = sum(areas)
    totalCount = len(areas)
    print('calculating area and count of warehouses in Riverside for year ', year)
    OutList.append([year, sumAreas, totalCount])
    
arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")

AreaByYear = np.array(OutList)
struct_array = numpy.core.records.fromarrays(
    AreaByYear.transpose(), numpy.dtype([('Year', '<i8'), ('AreaTotal', '<f8'), ('WarehousesCount', '<i8')]))
arcpy.da.NumPyArrayToTable(struct_array, data_path + r'\temp.gdb\AreaCountByYearRiverside')
m.addDataFromPath(data_path + r'\temp.gdb\AreaCountByYearRiverside')

AreaTable = m.listTables("AreaCountByYear*")[0]
delimfield = arcpy.AddFieldDelimiters(AreaTable, 'Year')

Warehouses has 948 records covering ~275465584 square feet of land in Riverside County.
calculating area and count of warehouses in Riverside for year  1975
calculating area and count of warehouses in Riverside for year  1980
calculating area and count of warehouses in Riverside for year  1985
calculating area and count of warehouses in Riverside for year  1990
calculating area and count of warehouses in Riverside for year  1995
calculating area and count of warehouses in Riverside for year  2000
calculating area and count of warehouses in Riverside for year  2005
calculating area and count of warehouses in Riverside for year  2010
calculating area and count of warehouses in Riverside for year  2015
calculating area and count of warehouses in Riverside for year  2020


In [11]:
arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
for year in ListOfYears:
    warehouseLyr.definitionQuery = "YEAR_BUILT <= " + year + " And YEAR_BUILT <> 0 And YEAR_BUILT <> 1776"
    with arcpy.da.SearchCursor(AreaTable, 'AreaTotal', where_clause=delimfield +' = ' + str(year)) as cursor:
        for row in cursor:
            totalArea = row[0]
            totalArea = round(totalArea)
    with arcpy.da.SearchCursor(AreaTable, 'WarehousesCount', where_clause=delimfield +' = ' + str(year)) as cursor:
        for row in cursor:
            warehousesCount = row[0]
    lytText.text = 'Distribution in ' + str(year)
    lytTextTotalArea.text = 'Area Occupied by Warehouses: '+ str(addCommas(totalArea)) + ' ft<sup>2</sup>'
    lytTextWarehouseCount.text = 'Total Number of Warehouses: ' + str(addCommas(warehousesCount))
    print('assessing year: ', year)
    arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
    lyt.exportToPDF(dst_path + r'\RCwarehouses' + str(year)) #PNG or SVG
    lyt.exportToSVG(dst_path + r'\RCwarehouses' + str(year)) #PNG or SVG
    lyt.exportToPNG(dst_path + r'\RCwarehouses' + str(year)) #PNG or SVG
arcpy.Delete_management(data_path + r'\temp.gdb\AreaCountByYearRiverside')
warehouseLyr.definitionQuery = None
lytText.text = 'Distribution in 2021'
lytTextTotalArea.text = 'Area Occupied by Warehouses: '+ str(addCommas(round(float(areaRiv)))) + ' ft<sup>2</sup>'
lytTextWarehouseCount.text = 'Total Number of Warehouses: ' + str(addCommas(int(countRiv[0])))
print('assessing Riverside County all entries.')
lyt.exportToPDF(dst_path + r'\RCwarehouses2021') #PNG or SVG
lyt.exportToSVG(dst_path + r'\RCwarehouses2021') #PNG or SVG
lyt.exportToPNG(dst_path + r'\RCwarehouses2021') #PNG or SVG


assessing year:  1975
assessing year:  1980
assessing year:  1985
assessing year:  1990
assessing year:  1995
assessing year:  2000
assessing year:  2005
assessing year:  2010
assessing year:  2015
assessing year:  2020
assessing Riverside County all entries.


'D:\\01_OntarioWarehouses\\InlandEmpireWarehouses\\maps\\RCwarehouses2021.png'

![Riverside County Warehouses in 2021](RCwarehouses2021.svg)

### After completing the county-level analyses, we repeat the same workflow to a merged shapefile of the two counties. 

In [84]:
####Initiate the the combined map environment and process fields to 
####create a single year attribute for parcels (not specific to its origin county)####

aprx = arcpy.mp.ArcGISProject("Current")
lyt = aprx.listLayouts("LayoutAll")[0]
m = aprx.listMaps("WarehousesAll")[0]
lytText = lyt.listElements("TEXT_ELEMENT", "YearsDisplayedText")[0]
lytTextTotalArea = lyt.listElements("TEXT_ELEMENT", "TotalArea")[0]
lytTextDisclaimer = lyt.listElements("TEXT_ELEMENT", "disclaimer")[0]
lytTextWarehouseCount = lyt.listElements("TEXT_ELEMENT", "WarehouseCount")[0]
logo = lyt.listElements("PICTURE_ELEMENT", 'RRC_Logo')[0]
logo.visible = True
northArrow = lyt.listElements("MAPSURROUND_ELEMENT",'North Arrow')[0]
####All warehouses merge###
arcpy.management.Merge(r"D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\derived\Warehouses_SanBernardino.shp;D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\derived\Warehouses_Riverside.shp", r"D:\01_OntarioWarehouses\InlandEmpireWarehouses\data\derived\Warehouses_Merged.shp", None, "NO_SOURCE_INFO")
warehouseLyr = m.listLayers('Warehouses_Merged*')[0]


In [85]:
#Updating symbology of the warehouse layer
warehouseLyr.name = 'Warehouses'
sym = warehouseLyr.symbology
sym.renderer.symbol.color = {'CMYK' : [0, 50, 100, 0, 100]}
sym.renderer.symbol.outlineColor = {'CMYK' : [0, 50, 100, 0, 0]}
warehouseLyr.symbology = sym

arcpy.AddField_management(warehouseLyr, "YEAR", "LONG")
arcpy.management.CalculateField(warehouseLyr, "YEAR", "foo(!BASE_YEAR!, !YEAR_BUILT!)", "PYTHON3", """def foo(sb, r):
    if r == 0 and sb != 0: 
        return sb 
    elif sb == 0 and (r !=0):
        return r
    else:
        return 0""", "TEXT", "NO_ENFORCE_DOMAINS")


<Result 'Warehouses'>

In [86]:
###Calculate total area and count of warehouses for the county###
warehouseLyr.definitionQuery = None
areaAll = sum([row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "Shape_Area")])
countAll = arcpy.management.GetCount(warehouseLyr)
print('{} has {} records covering ~{} square feet of land in the two counties.'.format(warehouseLyr, countAll[0], addCommas(areaAll)))

warehouseLyr.definitionQuery =  "YEAR = 0 Or YEAR = 1776"
areaZeroAll = sum([row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "Shape_Area")])
countZeroAll = arcpy.management.GetCount(warehouseLyr)
print('The {} data show {} records covering ~{} square feet of land in the two counties with no usable year datum.'.format(warehouseLyr, countZeroAll[0], addCommas(areaZeroAll)))
warehouseLyr.definitionQuery = None

Warehouses has 4299 records covering ~1,041,229,078.9496417 square feet of land in the two counties.
The Warehouses data show 314 records covering ~88,355,206.73399296 square feet of land in the two counties with no usable year datum.


In [87]:
OutList = []

totalCount = 0
sumAreas = 0

for year in ListOfYears:
    arcpy.management.SelectLayerByAttribute(warehouseLyr, 'NEW_SELECTION', "YEAR <= " + year + " And YEAR <> 0 And YEAR <> 1776")
    areas = [row[0] for row in arcpy.da.SearchCursor(warehouseLyr, "Shape_Area")]
    sumAreas = sum(areas)
    totalCount = len(areas)
    print('calculating area and count of warehouses in both counties for year ', year)
    print('sum areas is ',sumAreas,' and total count is ',totalCount)
    OutList.append([year, sumAreas, totalCount])
arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")

AreaByYear = np.array(OutList)
struct_array = numpy.core.records.fromarrays(
    AreaByYear.transpose(), numpy.dtype([('Year', '<i8'), ('AreaTotal', '<f8'), ('WarehousesCount', '<i8')]))
arcpy.da.NumPyArrayToTable(struct_array, data_path + r'\temp.gdb\AreaCountByYearAll')
m.addDataFromPath(data_path + r'\temp.gdb\AreaCountByYearAll')

AreaTable = m.listTables("AreaCountByYearAll*")[0]
delimfield = arcpy.AddFieldDelimiters(AreaTable, 'Year')

calculating area and count of warehouses in both counties for year  1975
sum areas is  14665553.726207228  and total count is  162
calculating area and count of warehouses in both counties for year  1980
sum areas is  22574120.742064003  and total count is  234
calculating area and count of warehouses in both counties for year  1985
sum areas is  31254458.03394447  and total count is  352
calculating area and count of warehouses in both counties for year  1990
sum areas is  56665852.62499533  and total count is  534
calculating area and count of warehouses in both counties for year  1995
sum areas is  87808390.19363551  and total count is  653
calculating area and count of warehouses in both counties for year  2000
sum areas is  168559472.25186104  and total count is  970
calculating area and count of warehouses in both counties for year  2005
sum areas is  300935721.3297937  and total count is  1574
calculating area and count of warehouses in both counties for year  2010
sum areas is 

In [88]:
arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
for year in ListOfYears:
    warehouseLyr.definitionQuery = "YEAR <= " + year + " And YEAR <> 0 And YEAR <> 1776"
    with arcpy.da.SearchCursor(AreaTable, 'AreaTotal', where_clause=delimfield +' = ' + str(year)) as cursor:
        for row in cursor:
            totalArea = row[0]
            totalArea = round(totalArea)
            print('gathering area: ', totalArea)
    with arcpy.da.SearchCursor(AreaTable, 'WarehousesCount', where_clause=delimfield +' = ' + str(year)) as cursor:
        for row in cursor:
            warehousesCount = row[0]
            print('gathering count:', row[0])
    lytText.text = 'Distribution in ' + str(year)
    lytTextTotalArea.text = 'Area Occupied by Warehouses: '+ str(addCommas(totalArea)) + ' ft<sup>2</sup>'
    lytTextWarehouseCount.text = 'Total Number of Warehouses: ' + str(addCommas(warehousesCount))
    print('assessing year: ', year)
    arcpy.SelectLayerByAttribute_management(warehouseLyr, "CLEAR_SELECTION")
    lyt.exportToPDF(dst_path + r'\SBRCwarehouses' + str(year)) #PNG or SVG
    lyt.exportToSVG(dst_path + r'\SBRCwarehouses' + str(year)) #PNG or SVG
    lyt.exportToPNG(dst_path + r'\SBRCwarehouses' + str(year)) #PNG or SVG
arcpy.Delete_management(data_path + r'\temp.gdb\AreaCountByYearAll')
warehouseLyr.definitionQuery = None
lytText.text = 'Distribution in 2021'
lytTextTotalArea.text = 'Area Occupied by Warehouses: '+ str(addCommas(round(float(areaAll)))) + ' ft<sup>2</sup>'
lytTextWarehouseCount.text = 'Total Number of Warehouses: ' + str(addCommas(int(countAll[0])))
print('assessing both counties all entries.')
updateNorthArrow(logo,northArrow)
lyt.exportToPDF(dst_path + r'\SBRCwarehouses2021_wLogo') #PNG or SVG
lyt.exportToSVG(dst_path + r'\SBRCwarehouses2021_wLogo') #PNG or SVG
lyt.exportToPNG(dst_path + r'\SBRCwarehouses2021_wLogo') #PNG or SVG
logo.visible=False
updateNorthArrow(logo,northArrow)
lyt.exportToPNG(dst_path + r'\SBRCwarehouses2021_noLogo') #PNG or SVG
lyt.exportToPDF(dst_path + r'\SBRCwarehouses2021_noLogo') #PNG or SVG
lyt.exportToSVG(dst_path + r'\SBRCwarehouses2021_noLogo') #PNG or SVG

gathering area:  14665554
gathering count: 162
assessing year:  1975
gathering area:  22574121
gathering count: 234
assessing year:  1980
gathering area:  31254458
gathering count: 352
assessing year:  1985
gathering area:  56665853
gathering count: 534
assessing year:  1990
gathering area:  87808390
gathering count: 653
assessing year:  1995
gathering area:  168559472
gathering count: 970
assessing year:  2000
gathering area:  300935721
gathering count: 1574
assessing year:  2005
gathering area:  406043081
gathering count: 2089
assessing year:  2010
gathering area:  590963478
gathering count: 2757
assessing year:  2015
gathering area:  888267256
gathering count: 3727
assessing year:  2020
assessing both counties all entries.


'D:\\01_OntarioWarehouses\\InlandEmpireWarehouses\\maps\\SBRCwarehouses2021_noLogo.svg'

![All County Warehouses in 2021](SBRCwarehouses2021_wLogo.svg)

#### To check the final data reductions using our workflow, a simple dictionary of values gathered along the way can be useful.

In [89]:
d = {'county' : ['Riverside','San Bernardino', 'all', 'all no years'], 'WarehouseCount' : [countRiv[0],countSB[0],countAll[0],countZeroAll[0]], 'WarehouseArea': [areaRiv,areaSB,areaAll,areaZeroAll]}
pd.DataFrame(d).to_csv(r"D:\01_OntarioWarehouses\WarehouseStudy\Riverside GIS Data\RiversideWarehouses\FinalWarehousesData.csv")